# Patients with multiple ICU stays

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#! UNCOMMENT BELOW IF USING ON COLAB
# Access data using Google BigQuery on Google Colab
# from google.colab import auth
# from google.cloud import bigquery
# auth.authenticate_user()

# Dataset paths
project_id = "composite-drive-359503"   #! CHANGEME
physionet_proj = "physionet-data"
mimic_core = f"{physionet_proj}.mimic_core"
mimic_hosp = f"{physionet_proj}.mimic_hosp"
mimic_icu = f"{physionet_proj}.mimic_icu"
mimic_derived = f"{physionet_proj}.mimic_derived"

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect='standard')

In [29]:
# Patients with multiple ICU stays
multiple_stays_query = f"""
--sql
SELECT
    icustays.subject_id AS subject_id,
    icustays.hadm_id AS hadm_id,
    count(icustays.stay_id) AS n_stays
FROM
    {mimic_icu}.icustays AS icustays
GROUP BY
    subject_id, hadm_id
HAVING
    n_stays > 1
ORDER BY
    subject_id, hadm_id
"""

In [30]:
run_query(multiple_stays_query)

,subject_id,hadm_id,n_stays
0,10002428,28662225,2
1,10003400,23559586,2
2,10006053,22942076,2
3,10011849,20188892,2
4,10012261,20719773,2
...,...,...,...
6128,19997293,20208898,2
6129,19997448,23560173,2
6130,19997473,27787494,2
6131,19998330,24096838,2


In [37]:
# ID patients with multiple ICU stays
patients_with_multiple_stays_query = f"""
--sql
SELECT
    *
FROM
    {mimic_icu}.icustays AS icustays
WHERE
    icustays.hadm_id IN (
        SELECT
            multiple_stays_pts.hadm_id
        FROM
            ({multiple_stays_query}) AS multiple_stays_pts
    )
ORDER BY
    subject_id, hadm_id
"""

In [38]:
run_query(patients_with_multiple_stays_query)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10002428,28662225,33987268,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-12 16:24:18,2156-04-17 15:57:08,4.981134
1,10002428,28662225,38875437,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894
2,10003400,23559586,38383343,Coronary Care Unit (CCU),Medical Intensive Care Unit (MICU),2137-08-17 17:36:37,2137-09-02 19:17:11,16.069838
3,10003400,23559586,34577403,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-08-10 19:54:51,2137-08-13 17:54:54,2.916701
4,10006053,22942076,32895909,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2111-11-13 23:40:00,2111-11-14 00:14:10,0.023727
...,...,...,...,...,...,...,...,...
13457,19997473,27787494,32568753,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2173-09-13 20:10:50,2173-09-17 11:37:50,3.643750
13458,19998330,24096838,31199714,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2178-11-29 21:51:19,2178-11-29 22:44:27,0.036898
13459,19998330,24096838,33428243,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2178-11-27 22:53:00,2178-11-29 21:29:39,1.942118
13460,19998591,24349193,31144045,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2185-07-16 18:48:18,2185-07-26 18:27:01,9.985220
